# BULK FEATURE EXTRACTION OF THE SYNTHETIC RV CURVES WITH `cesium`

In this notebook we do the bulk feature extraction with `cesium` for the RV curves in the synthetic databases (Datasets 1-4).

**IMPORTANT NOTE:** this code is probably not very efficient (for example, too many dataframe `append` operations, which is costly), but there is no special need at the moment to be more efficient. Maybe the solution is to create a 2D numpy array and then, at the end, create the DataFrame.

## Modules and configuration

### Modules

In [1]:
# Module import:
import warnings
import time

import pandas as pd
import numpy as np

from cesium.data_management import TimeSeries
from cesium.featurize import featurize_single_ts

### Configuration

In [2]:
SYNTH_FILE = "../data/RV_DATASETS/RV_All_GTO_SyntheticDatasets.csv"
RV_DS_FOLDER = "../data/RV_DATASETS/"

CS_FEATURES_FOLDER = "../data/DATASETS_CESIUM/"
OUT_DATASET_GEN_FILE = "cesium_DS<number>_Dataset.csv"

# LIST OF STAR METADATA TO ADD (DIFFERENT FOR EACH DATASET):
METADATA = {
    1: ['ID', 'Pulsating', 'frequency', 'amplitudeRV', 'offsetRV', 'refepochRV', 'phase',
        'D1_Ps', 'D1_Tobs'],
    2: ['ID', 'Pulsating', 'frequency', 'amplitudeRV', 'offsetRV', 'refepochRV', 'phase',
        'D1_Ps', 'D1_Tobs',
        'D2_noiseRV_mean', 'D2_noiseRV_median', 'D2_noiseRV_stdev'],
    3: ['ID', 'Pulsating', 'frequency', 'amplitudeRV', 'offsetRV', 'refepochRV', 'phase',
        'D3_samplingRV_idx', 'D3_PsRV_mean', 'D3_PsRV_median', 'D3_PsRV_stdev', 'D3_NumRV'],
    4: ['ID', 'Pulsating', 'frequency', 'amplitudeRV', 'offsetRV', 'refepochRV', 'phase',
        'D3_samplingRV_idx', 'D3_PsRV_mean', 'D3_PsRV_median', 'D3_PsRV_stdev', 'D3_NumRV',
        'D4_noiseRV_mean', 'D4_noiseRV_median', 'D4_noiseRV_stdev']
}

# A LIST OF ALL THE FEATURES CESIUM CAN EXTRACT (FOR REFERENCE PURPOSES)
ALL_CS_FEATURES = ['all_times_nhist_numpeaks',
                   'all_times_nhist_peak1_bin', 'all_times_nhist_peak2_bin', 'all_times_nhist_peak3_bin', 'all_times_nhist_peak4_bin',
                   'all_times_nhist_peak_1_to_2', 'all_times_nhist_peak_1_to_3', 'all_times_nhist_peak_1_to_4',
                   'all_times_nhist_peak_2_to_3', 'all_times_nhist_peak_2_to_4',
                   'all_times_nhist_peak_3_to_4',
                   'all_times_nhist_peak_val',
                   'avg_double_to_single_step', 'avg_err', 'avgt',
                   'cad_probs_1', 'cad_probs_10', 'cad_probs_20', 'cad_probs_30', 'cad_probs_40', 'cad_probs_50',
                   'cad_probs_100', 'cad_probs_500', 'cad_probs_1000', 'cad_probs_5000',
                   'cad_probs_10000', 'cad_probs_50000', 'cad_probs_100000', 'cad_probs_500000',
                   'cad_probs_1000000', 'cad_probs_5000000', 'cad_probs_10000000',
                   'cads_avg', 'cads_med', 'cads_std', 'mean',
                   'med_double_to_single_step', 'med_err',
                   'n_epochs', 'std_double_to_single_step', 'std_err',
                   'total_time', 'amplitude',
                   'flux_percentile_ratio_mid20', 'flux_percentile_ratio_mid35', 'flux_percentile_ratio_mid50',
                   'flux_percentile_ratio_mid65', 'flux_percentile_ratio_mid80',
                   'max_slope', 'maximum', 'median', 'median_absolute_deviation', 'minimum',
                   'percent_amplitude', 'percent_beyond_1_std', 'percent_close_to_median', 'percent_difference_flux_percentile',
                   'period_fast', 'qso_log_chi2_qsonu', 'qso_log_chi2nuNULL_chi2nu', 'skew', 'std',
                   'stetson_j', 'stetson_k', 'weighted_average', 'fold2P_slope_10percentile', 'fold2P_slope_90percentile',
                   'freq1_amplitude1', 'freq1_amplitude2', 'freq1_amplitude3', 'freq1_amplitude4',
                   'freq1_freq', 'freq1_lambda', 'freq1_rel_phase2', 'freq1_rel_phase3', 'freq1_rel_phase4', 'freq1_signif',
                   'freq2_amplitude1', 'freq2_amplitude2', 'freq2_amplitude3', 'freq2_amplitude4',
                   'freq2_freq', 'freq2_rel_phase2', 'freq2_rel_phase3', 'freq2_rel_phase4',
                   'freq3_amplitude1', 'freq3_amplitude2', 'freq3_amplitude3', 'freq3_amplitude4',
                   'freq3_freq', 'freq3_rel_phase2', 'freq3_rel_phase3', 'freq3_rel_phase4',
                   'freq_amplitude_ratio_21', 'freq_amplitude_ratio_31',
                   'freq_frequency_ratio_21', 'freq_frequency_ratio_31',
                   'freq_model_max_delta_mags', 'freq_model_min_delta_mags', 'freq_model_phi1_phi2',
                   'freq_n_alias', 'freq_signif_ratio_21', 'freq_signif_ratio_31',
                   'freq_varrat', 'freq_y_offset', 'linear_trend', 'medperc90_2p_p',
                   'p2p_scatter_2praw', 'p2p_scatter_over_mad', 'p2p_scatter_pfold_over_mad', 'p2p_ssqr_diff_over_var',
                   'scatter_res_raw']


## Load synthetic dataset information table

In [3]:
synth = pd.read_csv(SYNTH_FILE, sep=',', decimal='.')
synth.head()

,ID,Pulsating,frequency,amplitudeRV,offsetRV,refepochRV,phase,D1_Ps,D1_Tobs,D2_noiseRV_mean,...,D3_PsRV_median,D3_PsRV_stdev,D3_NumRV,D4_noiseRV_mean,D4_noiseRV_median,D4_noiseRV_stdev,ds1_file,ds2_file,ds3_file,ds4_file
0,RV-0,True,54.703173,1.133098,640.476258,2.457653e+06,0.737927,0.0025,0.25,0.244060,...,9.072850,83.352561,52,-0.416842,-0.659304,2.989282,./RV_DATASETS/DS1_ts_files/DS1-RV-RV-0.dat,./RV_DATASETS/DS2_ts_files/DS2-RV-RV-0.dat,./RV_DATASETS/DS3_ts_files/DS3-RV-RV-0.dat,./RV_DATASETS/DS4_ts_files/DS4-RV-RV-0.dat
1,RV-1,True,45.870515,0.945738,1392.825171,2.458556e+06,0.604099,0.0025,0.25,-0.064906,...,11.596005,161.691684,9,0.779823,1.449257,2.049067,./RV_DATASETS/DS1_ts_files/DS1-RV-RV-1.dat,./RV_DATASETS/DS2_ts_files/DS2-RV-RV-1.dat,./RV_DATASETS/DS3_ts_files/DS3-RV-RV-1.dat,./RV_DATASETS/DS4_ts_files/DS4-RV-RV-1.dat
2,RV-2,False,0.000000,0.000000,1799.268703,2.457479e+06,0.000000,0.0025,0.25,0.299835,...,17.867630,200.840308,26,-0.334425,-0.653148,2.803447,./RV_DATASETS/DS1_ts_files/DS1-RV-RV-2.dat,./RV_DATASETS/DS2_ts_files/DS2-RV-RV-2.dat,./RV_DATASETS/DS3_ts_files/DS3-RV-RV-2.dat,./RV_DATASETS/DS4_ts_files/DS4-RV-RV-2.dat
3,RV-3,False,0.000000,0.000000,918.007885,2.457612e+06,0.000000,0.0025,0.25,0.032798,...,24.896430,66.240066,6,0.854171,1.319561,3.021310,./RV_DATASETS/DS1_ts_files/DS1-RV-RV-3.dat,./RV_DATASETS/DS2_ts_files/DS2-RV-RV-3.dat,./RV_DATASETS/DS3_ts_files/DS3-RV-RV-3.dat,./RV_DATASETS/DS4_ts_files/DS4-RV-RV-3.dat
4,RV-4,True,69.503227,1.447038,-1375.321599,2.460177e+06,0.962170,0.0025,0.25,-0.403387,...,63.321765,303.823082,7,0.591993,-0.473597,3.343537,./RV_DATASETS/DS1_ts_files/DS1-RV-RV-4.dat,./RV_DATASETS/DS2_ts_files/DS2-RV-RV-4.dat,./RV_DATASETS/DS3_ts_files/DS3-RV-RV-4.dat,./RV_DATASETS/DS4_ts_files/DS4-RV-RV-4.dat


In [4]:
print(list(synth.columns))

['ID', 'Pulsating', 'frequency', 'amplitudeRV', 'offsetRV', 'refepochRV', 'phase', 'D1_Ps', 'D1_Tobs', 'D2_noiseRV_mean', 'D2_noiseRV_median', 'D2_noiseRV_stdev', 'D3_samplingRV_idx', 'D3_PsRV_mean', 'D3_PsRV_median', 'D3_PsRV_stdev', 'D3_NumRV', 'D4_noiseRV_mean', 'D4_noiseRV_median', 'D4_noiseRV_stdev', 'ds1_file', 'ds2_file', 'ds3_file', 'ds4_file']


## Feature extraction with `cesium` for the synthetic RV curves

In [6]:
# DISABLE WARNINGS:
warnings.filterwarnings('ignore')
# Batch processing:
lapse_list = []
median_lapse = None
# Initialize features dataframes and metafeatures (from disk, or new):
try:
    for ds in [1, 2, 3, 4]:
        df[ds] = pd.read_csv(CS_FEATURES_FOLDER + OUT_DATASET_GEN_FILE.replace("<number>", str(ds)),
                             sep=',', decimal='.')
        print("Previous result found in DS %d, will continue at record %d..." %(ds, len(df[ds])))
    i0=min(len(df[1]), len(df[2]), len(df[3]), len(df[4]))
    print("--- Will continue now at record %d..." %i0)
    # Restore all DataFrames to the minimum length:
    for ds in [1, 2, 3, 4]:
        df[ds] = df[ds][:i0].copy()

except:
    # No previous data stored in disk, initialize the DataFrames:
    print("No previous results found, initializing dataframe...")
    df = {
        1: None,
        2: None,
        3: None,
        4: None
    }
    i0=0
metadata_idx = METADATA
#for i in range(0, 3): # TEST
for i in range(i0, len(synth)):
    start_time = time.time()
    print("Record: %d, started at %s..."
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    for ds in [1, 2, 3, 4]:
        # For each dataset:
        # Get metafeatures values:
        metadata_values = list(synth.loc[i, metadata_idx[ds]])
        try:
            # load RV file:
            filename = RV_DS_FOLDER + \
                synth.loc[i, 'ds' + str(ds) + '_file'].replace("./RV_DATASETS/", "")
            rv = pd.read_csv(filename, sep=' ', decimal='.',
                             names=['time', 'rv'])
            # Create TimeSeries object:
            ts = TimeSeries(t=rv['time'], m=rv['rv'])
            # Featurize the time series:
            cs = featurize_single_ts(ts, features_to_use=ALL_CS_FEATURES)
            # Join metadata and features for the dataframe:
            indices = metadata_idx[ds] + ['VALID_RECORD'] + list(cs.index.get_level_values('feature'))
            values = metadata_values + [True] + list(cs.values)
        except Exception as e:
            # An exception was found, mark the record as invalid and set the features to 'nan':
            print("***ERROR: some error happened in record %d, dataset %d, " \
                  "marking the record as invalid. Error: %s"
                  %(i, ds, str(e)))
            indices = metadata_idx[ds] + ['VALID_RECORD'] + ALL_CS_FEATURES
            values = metadata_values + [False] + [np.nan] * 112
        if df[ds] is None:
            # Initialize DataFrame (with the first item):
            df[ds] = pd.DataFrame(data=[values], columns=indices)
        else:
            # Create a new DataFrame (with the new item):
            new_df = pd.DataFrame(data=[values], columns=indices)
            # Append the new dataframe to the existing one:
            df[ds] = df[ds].append(new_df, ignore_index=True)
        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        # Save the results:
        df[ds].to_csv(CS_FEATURES_FOLDER + OUT_DATASET_GEN_FILE.replace("<number>", str(ds)),
                      sep=',', decimal='.', index=False)


Previous result found in DS 1, will continue at record 213...
Previous result found in DS 2, will continue at record 213...
Previous result found in DS 3, will continue at record 212...
Previous result found in DS 4, will continue at record 212...
--- Will continue now at record 212...
Record: 212, started at 28/04/2022, 12:25:44...
Previous median lapse time: None
Record: 213, started at 28/04/2022, 12:25:45...
Previous median lapse time: 0.44 seconds
Record: 214, started at 28/04/2022, 12:25:46...
Previous median lapse time: 0.39 seconds
Record: 215, started at 28/04/2022, 12:25:47...
Previous median lapse time: 0.26 seconds
Record: 216, started at 28/04/2022, 12:25:48...
Previous median lapse time: 0.26 seconds
Record: 217, started at 28/04/2022, 12:25:49...
Previous median lapse time: 0.26 seconds
Record: 218, started at 28/04/2022, 12:25:49...
Previous median lapse time: 0.17 seconds
Record: 219, started at 28/04/2022, 12:25:49...
Previous median lapse time: 0.16 seconds
Record: 2

Record: 301, started at 28/04/2022, 12:26:51...
Previous median lapse time: 0.18 seconds
Record: 302, started at 28/04/2022, 12:26:53...
Previous median lapse time: 0.18 seconds
Record: 303, started at 28/04/2022, 12:26:55...
Previous median lapse time: 0.18 seconds
Record: 304, started at 28/04/2022, 12:26:56...
Previous median lapse time: 0.18 seconds
Record: 305, started at 28/04/2022, 12:26:57...
Previous median lapse time: 0.18 seconds
Record: 306, started at 28/04/2022, 12:26:59...
Previous median lapse time: 0.18 seconds
Record: 307, started at 28/04/2022, 12:27:00...
Previous median lapse time: 0.18 seconds
Record: 308, started at 28/04/2022, 12:27:00...
Previous median lapse time: 0.18 seconds
Record: 309, started at 28/04/2022, 12:27:01...
Previous median lapse time: 0.18 seconds
Record: 310, started at 28/04/2022, 12:27:04...
Previous median lapse time: 0.18 seconds
Record: 311, started at 28/04/2022, 12:27:04...
Previous median lapse time: 0.18 seconds
Record: 312, started 

Record: 394, started at 28/04/2022, 12:28:19...
Previous median lapse time: 0.18 seconds
Record: 395, started at 28/04/2022, 12:28:19...
Previous median lapse time: 0.18 seconds
Record: 396, started at 28/04/2022, 12:28:19...
Previous median lapse time: 0.18 seconds
Record: 397, started at 28/04/2022, 12:28:20...
Previous median lapse time: 0.18 seconds
Record: 398, started at 28/04/2022, 12:28:21...
Previous median lapse time: 0.18 seconds
Record: 399, started at 28/04/2022, 12:28:23...
Previous median lapse time: 0.18 seconds
Record: 400, started at 28/04/2022, 12:28:24...
Previous median lapse time: 0.18 seconds
Record: 401, started at 28/04/2022, 12:28:25...
Previous median lapse time: 0.18 seconds
Record: 402, started at 28/04/2022, 12:28:25...
Previous median lapse time: 0.19 seconds
Record: 403, started at 28/04/2022, 12:28:26...
Previous median lapse time: 0.19 seconds
Record: 404, started at 28/04/2022, 12:28:26...
Previous median lapse time: 0.19 seconds
Record: 405, started 

Record: 487, started at 28/04/2022, 12:29:34...
Previous median lapse time: 0.22 seconds
Record: 488, started at 28/04/2022, 12:29:35...
Previous median lapse time: 0.22 seconds
Record: 489, started at 28/04/2022, 12:29:37...
Previous median lapse time: 0.22 seconds
Record: 490, started at 28/04/2022, 12:29:38...
Previous median lapse time: 0.22 seconds
Record: 491, started at 28/04/2022, 12:29:39...
Previous median lapse time: 0.22 seconds
Record: 492, started at 28/04/2022, 12:29:40...
Previous median lapse time: 0.22 seconds
Record: 493, started at 28/04/2022, 12:29:41...
Previous median lapse time: 0.22 seconds
Record: 494, started at 28/04/2022, 12:29:42...
Previous median lapse time: 0.22 seconds
Record: 495, started at 28/04/2022, 12:29:43...
Previous median lapse time: 0.22 seconds
Record: 496, started at 28/04/2022, 12:29:45...
Previous median lapse time: 0.22 seconds
Record: 497, started at 28/04/2022, 12:29:45...
Previous median lapse time: 0.22 seconds
Record: 498, started 

Record: 580, started at 28/04/2022, 12:30:57...
Previous median lapse time: 0.22 seconds
Record: 581, started at 28/04/2022, 12:30:57...
Previous median lapse time: 0.22 seconds
Record: 582, started at 28/04/2022, 12:30:59...
Previous median lapse time: 0.22 seconds
Record: 583, started at 28/04/2022, 12:31:02...
Previous median lapse time: 0.22 seconds
Record: 584, started at 28/04/2022, 12:31:02...
Previous median lapse time: 0.22 seconds
Record: 585, started at 28/04/2022, 12:31:04...
Previous median lapse time: 0.22 seconds
Record: 586, started at 28/04/2022, 12:31:06...
Previous median lapse time: 0.22 seconds
Record: 587, started at 28/04/2022, 12:31:07...
Previous median lapse time: 0.22 seconds
Record: 588, started at 28/04/2022, 12:31:09...
Previous median lapse time: 0.22 seconds
Record: 589, started at 28/04/2022, 12:31:09...
Previous median lapse time: 0.22 seconds
Record: 590, started at 28/04/2022, 12:31:10...
Previous median lapse time: 0.22 seconds
Record: 591, started 

Record: 673, started at 28/04/2022, 12:32:34...
Previous median lapse time: 0.23 seconds
Record: 674, started at 28/04/2022, 12:32:35...
Previous median lapse time: 0.23 seconds
Record: 675, started at 28/04/2022, 12:32:36...
Previous median lapse time: 0.23 seconds
Record: 676, started at 28/04/2022, 12:32:36...
Previous median lapse time: 0.23 seconds
Record: 677, started at 28/04/2022, 12:32:37...
Previous median lapse time: 0.23 seconds
Record: 678, started at 28/04/2022, 12:32:38...
Previous median lapse time: 0.23 seconds
Record: 679, started at 28/04/2022, 12:32:39...
Previous median lapse time: 0.23 seconds
Record: 680, started at 28/04/2022, 12:32:39...
Previous median lapse time: 0.23 seconds
Record: 681, started at 28/04/2022, 12:32:40...
Previous median lapse time: 0.23 seconds
Record: 682, started at 28/04/2022, 12:32:41...
Previous median lapse time: 0.23 seconds
Record: 683, started at 28/04/2022, 12:32:42...
Previous median lapse time: 0.23 seconds
Record: 684, started 

Record: 766, started at 28/04/2022, 12:34:52...
Previous median lapse time: 0.23 seconds
Record: 767, started at 28/04/2022, 12:34:53...
Previous median lapse time: 0.23 seconds
Record: 768, started at 28/04/2022, 12:34:54...
Previous median lapse time: 0.23 seconds
Record: 769, started at 28/04/2022, 12:34:55...
Previous median lapse time: 0.23 seconds
Record: 770, started at 28/04/2022, 12:34:57...
Previous median lapse time: 0.23 seconds
Record: 771, started at 28/04/2022, 12:34:57...
Previous median lapse time: 0.23 seconds
Record: 772, started at 28/04/2022, 12:34:58...
Previous median lapse time: 0.23 seconds
Record: 773, started at 28/04/2022, 12:34:59...
Previous median lapse time: 0.23 seconds
Record: 774, started at 28/04/2022, 12:34:59...
Previous median lapse time: 0.23 seconds
Record: 775, started at 28/04/2022, 12:35:00...
Previous median lapse time: 0.23 seconds
Record: 776, started at 28/04/2022, 12:35:01...
Previous median lapse time: 0.23 seconds
Record: 777, started 

Record: 859, started at 28/04/2022, 12:36:25...
Previous median lapse time: 0.23 seconds
Record: 860, started at 28/04/2022, 12:36:25...
Previous median lapse time: 0.23 seconds
Record: 861, started at 28/04/2022, 12:36:27...
Previous median lapse time: 0.23 seconds
Record: 862, started at 28/04/2022, 12:36:28...
Previous median lapse time: 0.23 seconds
Record: 863, started at 28/04/2022, 12:36:29...
Previous median lapse time: 0.23 seconds
Record: 864, started at 28/04/2022, 12:36:29...
Previous median lapse time: 0.23 seconds
Record: 865, started at 28/04/2022, 12:36:31...
Previous median lapse time: 0.23 seconds
Record: 866, started at 28/04/2022, 12:36:32...
Previous median lapse time: 0.23 seconds
Record: 867, started at 28/04/2022, 12:36:32...
Previous median lapse time: 0.23 seconds
Record: 868, started at 28/04/2022, 12:36:33...
Previous median lapse time: 0.23 seconds
Record: 869, started at 28/04/2022, 12:36:34...
Previous median lapse time: 0.23 seconds
Record: 870, started 

Record: 952, started at 28/04/2022, 12:38:19...
Previous median lapse time: 0.24 seconds
Record: 953, started at 28/04/2022, 12:38:20...
Previous median lapse time: 0.24 seconds
Record: 954, started at 28/04/2022, 12:38:21...
Previous median lapse time: 0.24 seconds
Record: 955, started at 28/04/2022, 12:38:21...
Previous median lapse time: 0.24 seconds
Record: 956, started at 28/04/2022, 12:38:22...
Previous median lapse time: 0.24 seconds
Record: 957, started at 28/04/2022, 12:38:23...
Previous median lapse time: 0.24 seconds
Record: 958, started at 28/04/2022, 12:38:24...
Previous median lapse time: 0.24 seconds
Record: 959, started at 28/04/2022, 12:38:25...
Previous median lapse time: 0.24 seconds
Record: 960, started at 28/04/2022, 12:38:26...
Previous median lapse time: 0.24 seconds
Record: 961, started at 28/04/2022, 12:38:27...
Previous median lapse time: 0.24 seconds
Record: 962, started at 28/04/2022, 12:38:28...
Previous median lapse time: 0.24 seconds
Record: 963, started 

### Next steps are to be executed only if the cell execution is user-interrupted

For example, if the user decided to interrupt the cell execution because it got stuck in some record, the next cells update the info for that record with an "invalid record" mark.

Afterwards, the loop (previous cell) can be executed again and it will start from the record following the problematic one.

In [ ]:
i

In [ ]:
df[1].tail()

In [ ]:
synth.loc[i]

In [ ]:
# Update the wrong record:
for ds in [1 , 2, 3, 4]:
    indices = metadata_idx[ds] + ['VALID_RECORD'] + ALL_CS_FEATURES
    metadata_values = list(synth.loc[i, metadata_idx[ds]])
    values = metadata_values + [False] + [np.nan] * 112
    new_df = pd.DataFrame(data=[values], columns=indices)
    df[ds] = df[ds].append(new_df, ignore_index=True)


In [ ]:
df[1].tail()

In [ ]:
# Save the results:
for ds in [1, 2, 3, 4]
    df[ds].to_csv(CS_FEATURES_FOLDER + OUT_DATASET_GEN_FILE.replace("<number>", str(ds)),
                  sep=',', decimal='.', index=False)


In [ ]:
df[1].head()

## Review the records with errors

In [7]:
df[1][df[1]['VALID_RECORD'] == False]

,ID,Pulsating,frequency,amplitudeRV,offsetRV,refepochRV,phase,D1_Ps,D1_Tobs,VALID_RECORD,...,freq_signif_ratio_31,freq_varrat,freq_y_offset,linear_trend,medperc90_2p_p,p2p_scatter_2praw,p2p_scatter_over_mad,p2p_scatter_pfold_over_mad,p2p_ssqr_diff_over_var,scatter_res_raw


## Summary

**CONCLUSIONS:**
- Completed the `cesium` feature extraction of the four synthetic dataset objects.
- No problems were found in calculations.